In [ ]:
import asyncio
from IPython.display import display
from ipywidgets import HTML, Output

outputs = [Output() for _ in range(3)]
display(*outputs)

async def reproduce_bug():
    for i in range(3):
        await asyncio.sleep(0.01)
        outputs[i].append_display_data(HTML(f"Content {i}"))

asyncio.create_task(reproduce_bug())

In [ ]:
# Test synchronous append_display_data (per comment from DonJayamanne)
sync_outputs = [Output() for _ in range(2)]
display(*sync_outputs)

In [ ]:
sync_outputs[0].append_display_data(HTML("Sync Content 0"))
sync_outputs[1].append_display_data(HTML("Sync Content 1"))